In [47]:
import nltk
import requests
import warnings
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime

### Aux Functions 

In [294]:
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\T-Gamer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\T-Gamer\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


# 1.0. CF88 Data Collect

## Data Collect

In [68]:
hdr = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
#url = 'https://www2.camara.leg.br/legin/fed/consti/1988/constituicao-1988-5-outubro-1988-322142-publicacaooriginal-1-pl.html'
#url = "https://www.planalto.gov.br/ccivil_03/constituicao/constituicao.htm"

#pg = requests.get(url=url, headers=hdr)
#soup = BeautifulSoup(pg.text, 'html.parser')

soup = BeautifulSoup(open('page/Constituição.html').read(), 'html.parser')

In [76]:
#source = soup.title.get_text()
source = ' '.join([k.get_text().strip() for k in soup.find_all('big')][1:])

#preambulo = soup.find('p', class_='preambulo').get_text()

In [81]:
# soup.find('div', attrs={'id': 'art'})

In [77]:
# Pre-Processing
txt = soup.find('div', class_='textoNorma').get_text().replace('rt.', 'rt').replace('º', '.').replace('arts.', 'arts ')

pre_txt = [k.strip() for k in nltk.tokenize.sent_tokenize(
    txt, language='portuguese'
)]

pre_txt = [list(filter(None, r.split('\xa0'))) for r in pre_txt]

AttributeError: 'NoneType' object has no attribute 'get_text'

In [51]:
# Fix Titles
aux = []
for r in pre_txt:
    if r[0].startswith('TÍTULO'):
        aux.append(r[:-1][0].split('\n')[:2])
        aux.append(r[-1:])
    elif r[0].startswith('CAP'):
        aux.append([r[-1]])
    elif len(r) > 1 and r[-1].upper().startswith('ART'):
        aux.append(r[:-1])
        aux.append([r[-1]])
    else:
        aux.append(r)

aux = pd.Series(aux).drop_duplicates().tolist()

In [52]:
# Create Pandas DataFrame.
a = []
for i in range(0, len(aux[1:])):
    r = aux[1:][i]
    
    if r[0].startswith('Art'):
        a.append(
            {'artigo': ' '.join(r + aux[1:][i+1])}
        )
    
    elif r[0].startswith('Parágrafo único'):
        a.append(
            {'paragrafo_unico': ' '.join(r + aux[1:][i+1])}
        )
    
    elif r[0].startswith('§'):
        a.append(
            {'paragrafo': ' '.join(r + aux[1:][i+1])}
        )
    
    elif r[0].startswith('TÍTULO'):
        a.append(
            {'titulo': ' '.join(r)}
        )

for row in a:
    df = pd.DataFrame(a)

In [58]:
df.artigo[df.artigo.fillna('na').str.contains('Art 102.')].to_list()

['Art 102. Compete ao Supremo Tribunal Federal, precipuamente, a guarda da Constituição, cabendo-lhe:   I - processar e julgar, originariamente: \n\n\n\n\na)\na ação direta de inconstitucionalidade de lei ou ato normativo federal ou estadual; \n\n\n\n\nb)\nnas infrações penais comuns, o Presidente da República, o Vice-Presidente, os membros do Congresso Nacional, seus próprios Ministros e o Procurador-Geral da República; \n\n\n\n\nc)\nnas infrações penais comuns e nos crimes de responsabilidade, os Ministros de Estado, ressalvado o disposto no art 52, I, os membros dos Tribunais Superiores, os do Tribunal de Contas da União e os chefes de missão diplomática de caráter permanente; \n\n\n\n\nd)\no habeas corpus , sendo paciente qualquer das pessoas referidas nas alíneas anteriores; o mandado de segurança e o habeas data contra atos do Presidente da República, das Mesas da Câmara dos Deputados e do Senado Federal, do Tribunal de Contas da União, do Procurador-Geral da República e do própr

## Data Cleaning

In [59]:
alineas_list = [
    'a)', 'b)', 'c)', 'd)', 'e)', 'f)', 'g)', 'h)', 'i)', 'j)', 'k)', 'l)', 'm)',
    'n)', 'o)', 'p)', 'q)', 'r)', 's)', 't)', 'u)', 'v)', 'w)', 'x)', 'y)', 'z)'
]

base_dict = {
    'sigla': '', 'livro': '', 'titulo': '',
    'capitulo': '', 'sessao': '', 'subsessao': '',
    'artigo': '', 'paragrafo': '', 'incisos': '',
    'alineas': '', 'lei': '', 'scrapy_datetime': '', 'link': '',
}

In [60]:
df.artigo = [k.replace(':', ";") if pd.notna(k) else k for k in df.artigo.tolist()]
df.paragrafo = df.paragrafo.apply(lambda x: x.replace(':', ';') if pd.notnull(x) else x)

In [10]:
df.head(2)

,titulo,artigo,paragrafo_unico,paragrafo
0,TÍTULO I DOS PRINCÍPIOS FUNDAMENTAIS,NaN,NaN,NaN
1,NaN,"Art 1. A República Federativa do Brasil, forma...",NaN,NaN


### Arts Cleaning 

In [11]:
df1 = df.artigo.str.split(';', expand=True)

df_arts = pd.DataFrame(columns=['body', 'artigo', 'incisos'])

In [12]:
for col in range(0, df1.T.shape[-1]):
    df2 = df1.T[col]

    aux = df2[pd.notna(df2)]

    if not aux.empty:
        art = aux[0].split('.')[0]

        art_body = {"body": aux[0].split('.')[1]}

        df_art = pd.DataFrame(art_body, index=[0])

        df_art['artigo'] = art

        df_i = pd.DataFrame([i.strip().split(' - ') for i in aux[1:]]).rename(columns={0:"incisos", 1:"body"})
        df_i['artigo'] = art

        df_clean = pd.concat([df_art, df_i], axis=0).reset_index(drop=True)

        df_arts = pd.concat([df_arts, df_clean], axis=0)

df_arts.incisos = df_arts.incisos.fillna(0)

df_arts = df_arts.reset_index(drop=True)

In [13]:
alineas_index = df_arts['incisos'].str.extract("(^[a-z]\)?)").dropna()[0].index

alineas = df_arts.loc[df_arts.incisos.index.isin(alineas_index), "incisos"]
alineas_body = pd.Series({k:v[5:].capitalize() for k, v in zip(alineas_index, alineas.to_list())})
alineas_body.name = "body"

alineas_simbol = alineas.str.extract("([a-z]\))")[0]

alineas_simbol.name = "alineas"

df_arts = pd.concat([df_arts, alineas_simbol], axis=1)

df_arts.body = df_arts.body.combine_first(alineas_body)

df_arts.loc[df_arts.incisos.index.isin(alineas_index), "incisos"] = np.nan

df_arts.incisos = df_arts.incisos.fillna(method="ffill")

In [14]:
df_arts.head(3)

,body,artigo,incisos,alineas
0,"A República Federativa do Brasil, formada pel...",Art 1,0,NaN
1,a soberania,Art 1,I,NaN
2,a cidadania,Art 1,II,NaN


In [15]:
df_arts['incisos'] = df_arts['incisos'].replace(0, "na")
df_arts['alineas'] = df_arts['alineas'].fillna("na")

In [16]:
df_ref = pd.DataFrame(base_dict, index=[0])

### Title Cleaning

In [17]:
df2 = df_arts.copy()
df2['titulo'] = np.nan

aux = df[['titulo', 'artigo']].fillna("NaN")

In [18]:
title_index = aux[~aux['titulo'].str.contains('NaN')]['titulo'].index
title_arts = aux.iloc[(aux.titulo.index.isin(title_index)|(aux.artigo.index.isin(title_index+1))), :].replace("NaN", np.nan)

title_arts = pd.concat([title_arts['titulo'].dropna().reset_index(drop=True), title_arts['artigo'].dropna().reset_index(drop=True)], axis=1)

title_arts['artigo'] = title_arts['artigo'].str.extract('(^[A-z]+\s\d+)')[0].tolist()

In [19]:
# Search title location index on DF2
aux = df2.artigo.drop_duplicates()
title_index = aux[aux.isin(title_arts['artigo'])]

df2.iloc[title_index.index, -1] = title_arts['titulo'].to_list() 

df2['titulo'] = df2['titulo'].fillna(method="ffill")

In [20]:
# Dataset Preparation
df2 = df2.rename(columns={"body":"lei"})

df2['lei'] = df2['lei'].apply(lambda x: x.strip().capitalize())

In [21]:
token_titles = [nltk.tokenize.word_tokenize(k) for k in df2['titulo'].to_list()]
df2['titulo'] = [' '.join(k[:2] + ['-'] + k[2:]) for k in token_titles]

In [22]:
df2.head(3)

,lei,artigo,incisos,alineas,titulo
0,"A república federativa do brasil, formada pela...",Art 1,na,na,TÍTULO I - DOS PRINCÍPIOS FUNDAMENTAIS
1,A soberania,Art 1,I,na,TÍTULO I - DOS PRINCÍPIOS FUNDAMENTAIS
2,A cidadania,Art 1,II,na,TÍTULO I - DOS PRINCÍPIOS FUNDAMENTAIS


### Parag Cleaning

In [23]:
df3 = df2.copy()

In [24]:
aux = df[['artigo', 'paragrafo_unico']]#.fillna("NaN")
aux = aux.iloc[1:, :].reset_index(drop=True)

aux['artigo'] = aux['artigo'].fillna(method='ffill')

aux = aux.dropna().reset_index(drop=True)
aux = aux.rename(columns={'artigo': 'lei'})

aux['artigo'] = aux['lei'].str.extract('([a-zA-Z ]+\d+)').rename(columns={0: 'artigo'})

aux = aux[['artigo', 'paragrafo_unico']]

aux['paragrafo_unico'] = aux.paragrafo_unico.apply(lambda x: nltk.sent_tokenize(x)[1])

In [25]:
aux.columns = ['artigo', 'paragrafo']

In [26]:
aux.head(2)

,artigo,paragrafo
0,Art 1,"Todo o poder emana do povo, que o exerce por m..."


In [27]:
df3.index = range(0, 2*len(df3), 2)
df3 = df3.reindex(index=range(len(df3)*2))

In [28]:
df3['artigo'] = df3['artigo'].fillna(method='ffill')

df3 = pd.merge(df3, aux, on='artigo', how='left')

df3['lei'] = df3.apply(lambda x: x['paragrafo'] if pd.isna(x['lei']) else x['lei'], axis=1)

df3 = df3.drop_duplicates()

df3['paragrafo'] = df3.apply(lambda x: 
                        'titulo' if x['incisos']=='na' and x['alineas']=='na' else 
                        'unico'  if pd.isna(x['titulo']) else 'na', axis=1)

df3 = df3.drop_duplicates()

df3 = df3[~df3['lei'].isna()].fillna(method='ffill')

df3 = df3.reset_index(drop=True)

In [29]:
df3['sigla'] = 'CF88'
df3['link']  = 'https://www.planalto.gov.br/ccivil_03/constituicao/constituicaocompilado.htm'
df3['scrapy_datetime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
df3['process_datetime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

df3 = df3[['sigla', 'titulo', 'lei', 'artigo', 'incisos', 'alineas', 'paragrafo',
           'link', 'scrapy_datetime', 'process_datetime']]

In [30]:
df3.head(2)

,sigla,titulo,lei,artigo,incisos,alineas,paragrafo,link,scrapy_datetime,process_datetime
0,CF88,TÍTULO I - DOS PRINCÍPIOS FUNDAMENTAIS,"A república federativa do brasil, formada pela...",Art 1,na,na,titulo,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
1,CF88,TÍTULO I - DOS PRINCÍPIOS FUNDAMENTAIS,"Todo o poder emana do povo, que o exerce por m...",Art 1,na,na,unico,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54


In [31]:
df3.to_csv('../data/cf88_27-08-2022.csv', index=False)

## Dataset Verify 

In [32]:
df1 = pd.read_csv('../data/cf88_27-08-2022.csv')

In [33]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127 entries, 0 to 1126
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   sigla             1127 non-null   object
 1   titulo            1127 non-null   object
 2   lei               1127 non-null   object
 3   artigo            1127 non-null   object
 4   incisos           1127 non-null   object
 5   alineas           1127 non-null   object
 6   paragrafo         1127 non-null   object
 7   link              1127 non-null   object
 8   scrapy_datetime   1127 non-null   object
 9   process_datetime  1127 non-null   object
dtypes: object(10)
memory usage: 88.2+ KB


### Arts Verify

In [34]:
df1.artigo.unique()

array(['Art 1', 'Art 2', 'Art 3', 'Art 4', 'Art 5', 'Art 6', 'Art 7',
       'Art 8', 'Art 9', 'Art 10', 'Art 11', 'Art 12', 'Art 13', 'Art 14',
       'Art 15', 'Art 16', 'Art 17', 'Art 18', 'Art 19', 'Art 20',
       'Art 21', 'Art 22', 'Art 23', 'Art 24', 'Art 25', 'Art 26',
       'Art 27', 'Art 28', 'Art 29', 'Art 30', 'Art 31', 'Art 32',
       'Art 33', 'Art 34', 'Art 35', 'Art 36', 'Art 37', 'Art 38',
       'Art 39', 'Art 40', 'Art 41', 'Art 42', 'Art 43', 'Art 44',
       'Art 45', 'Art 46', 'Art 47', 'Art 48', 'Art 49', 'Art 50',
       'Art 51', 'Art 52', 'Art 53', 'Art 54', 'Art 55', 'Art 56',
       'Art 57', 'Art 58', 'Art 59', 'Art 60', 'Art 61', 'Art 62',
       'Art 63', 'Art 64', 'Art 65', 'Art 66', 'Art 67', 'Art 68',
       'Art 69', 'Art 70', 'Art 71', 'Art 72', 'Art 73', 'Art 74',
       'Art 75', 'Art 76', 'Art 77', 'Art 78', 'Art 79', 'Art 80',
       'Art 81', 'Art 82', 'Art 83', 'Art 84', 'Art 85', 'Art 86',
       'Art 87', 'Art 88', 'Art 89', 'Art 90', 'Art

In [40]:
df1[df1.artigo == 'Art 15']

,sigla,titulo,lei,artigo,incisos,alineas,paragrafo,link,scrapy_datetime,process_datetime
189,CF88,TÍTULO II - DOS DIREITOS E GARANTIAS FUNDAMENTAIS,"É vedada a cassação de direitos políticos, cuj...",Art 15,na,na,titulo,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
190,CF88,TÍTULO II - DOS DIREITOS E GARANTIAS FUNDAMENTAIS,Cancelamento da naturalização por sentença tra...,Art 15,I,na,na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
191,CF88,TÍTULO II - DOS DIREITOS E GARANTIAS FUNDAMENTAIS,Incapacidade civil absoluta,Art 15,II,na,na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
192,CF88,TÍTULO II - DOS DIREITOS E GARANTIAS FUNDAMENTAIS,"Condenação criminal transitada em julgado, enq...",Art 15,III,na,na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
193,CF88,TÍTULO II - DOS DIREITOS E GARANTIAS FUNDAMENTAIS,Recusa de cumprir obrigação a todos imposta ou...,Art 15,IV,na,na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
194,CF88,TÍTULO II - DOS DIREITOS E GARANTIAS FUNDAMENTAIS,"Improbidade administrativa, nos termos do art ...",Art 15,V,na,na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54


In [38]:
df1[df1.artigo == 'Art 15']['lei'].tolist()

['É vedada a cassação de direitos políticos, cuja perda ou suspensão só se dará nos casos de',
 'Cancelamento da naturalização por sentença transitada em julgado',
 'Incapacidade civil absoluta',
 'Condenação criminal transitada em julgado, enquanto durarem seus efeitos',
 'Recusa de cumprir obrigação a todos imposta ou prestação alternativa, nos termos do art 5., viii',
 'Improbidade administrativa, nos termos do art 37, § 4..']

In [43]:
df1[df1.artigo == 'Art 16']

,sigla,titulo,lei,artigo,incisos,alineas,paragrafo,link,scrapy_datetime,process_datetime
195,CF88,TÍTULO II - DOS DIREITOS E GARANTIAS FUNDAMENTAIS,A lei que alterar o processo eleitoral só entr...,Art 16,na,na,titulo,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54


In [44]:
df1[df1.artigo == 'Art 16']['lei'].tolist()

['A lei que alterar o processo eleitoral só entrará em vigor um ano após sua promulgação']

In [46]:
df1[df1.artigo == 'Art 102']

,sigla,titulo,lei,artigo,incisos,alineas,paragrafo,link,scrapy_datetime,process_datetime
666,CF88,TÍTULO IV - DA ORGANIZAÇÃO DOS PODERES,"Compete ao supremo tribunal federal, precipuam...",Art 102,na,na,titulo,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
667,CF88,TÍTULO IV - DA ORGANIZAÇÃO DOS PODERES,"Processar e julgar, originariamente",Art 102,I,na,na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
668,CF88,TÍTULO IV - DA ORGANIZAÇÃO DOS PODERES,Ação direta de inconstitucionalidade de lei ou...,Art 102,I,a),na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
669,CF88,TÍTULO IV - DA ORGANIZAÇÃO DOS PODERES,"S infrações penais comuns, o presidente da rep...",Art 102,I,b),na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
670,CF88,TÍTULO IV - DA ORGANIZAÇÃO DOS PODERES,S infrações penais comuns e nos crimes de resp...,Art 102,I,c),na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
671,CF88,TÍTULO IV - DA ORGANIZAÇÃO DOS PODERES,"Habeas corpus , sendo paciente qualquer das pe...",Art 102,I,d),na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
672,CF88,TÍTULO IV - DA ORGANIZAÇÃO DOS PODERES,Dado de segurança e o habeas data contra atos ...,Art 102,I,na,na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
673,CF88,TÍTULO IV - DA ORGANIZAÇÃO DOS PODERES,Litígio entre estado estrangeiro ou organismo ...,Art 102,I,e),na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
674,CF88,TÍTULO IV - DA ORGANIZAÇÃO DOS PODERES,Causas e os conflitos entre a união e os estad...,Art 102,I,f),na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
675,CF88,TÍTULO IV - DA ORGANIZAÇÃO DOS PODERES,Extradição solicitada por estado estrangeiro,Art 102,I,g),na,https://www.planalto.gov.br/ccivil_03/constitu...,2022-08-27 12:36:54,2022-08-27 12:36:54
